# General Eye localization Dataset Processing pipeline

### Standard Library Imports

In [1]:
import sys, os
import hdf5storage
import numpy as np
import shutil
from statistics import mean
from pathlib import Path


### Local Module Imports

In [2]:
sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

from src.utils.paths_manager import LFPW_RAW_SOURCE_PATH, HELEN_RAW_SOURCE_PATH
from src.dataset.loaders import LFPWDataset, HelenDataset, FaceLandmarkDataloaderContext
from src.dataset.ops import FaceLandmarkImageOps
from src.utils.data_structs import ColumnStruct

### Instantiate Main Context class

In [3]:
dloader_ctx = FaceLandmarkDataloaderContext()

In [4]:
# Dataset source container
dataset_container = [
    LFPWDataset(LFPW_RAW_SOURCE_PATH),
    HelenDataset(HELEN_RAW_SOURCE_PATH)
]

# Tset container 
tset_container = [
    'test',
    'train'
]

In [5]:
# Dataset processing settings
DATASETS_PATH = Path('../datasets')

DESTINATION_PATH = DATASETS_PATH / 'processed' / 'MIRRORED_ITERATION'
TEST_DESTINATION_PATH = DESTINATION_PATH / 'test'
TRAIN_DESTINATION_PATH = DESTINATION_PATH / 'train'

if not DESTINATION_PATH.exists():
    DESTINATION_PATH.mkdir()

if not TEST_DESTINATION_PATH.exists():
    TEST_DESTINATION_PATH.mkdir()

if not TRAIN_DESTINATION_PATH.exists():
    TRAIN_DESTINATION_PATH.mkdir()

In [6]:
shutil.copy('../extras/test/see_bboxes.m', str(TEST_DESTINATION_PATH/'see_bboxes.m'))
shutil.copy('../extras/train/see_bboxes.m', str(TRAIN_DESTINATION_PATH/'see_bboxes.m'))

'..\\datasets\\processed\\MIRRORED_ITERATION\\train\\see_bboxes.m'

In [7]:
STANDARD_FACE_WIDTH = 300
STANDARD_FACE_HEIGHT = 420

In [8]:
combined_container = []

for tset in tset_container:
    for dset in dataset_container:
        dloader_ctx.dloader = dset
        if not dloader_ctx.check_valid():
            print('Invalid Dataset Loader')
            break

        for fl_image in dloader_ctx.load(tset):
            
            orig_fname = os.path.basename(fl_image.fpath)

            # Scale image
            leftmost = min([x[0] for x in fl_image.landmarks.face])
            rightmost = max([x[0] for x in fl_image.landmarks.face])

            face_width = rightmost - leftmost
            scaling_factor = STANDARD_FACE_WIDTH / face_width

            im = fl_image.uniform_scale(scaling_factor)

            # Crop image
            leftmost = min([x[0] for x in im.landmarks.face])
            rightmost = max([x[0] for x in im.landmarks.face])

            face_outline_cluster = im.landmarks.face + im.landmarks.left_brow + im.landmarks.right_brow
            y_median = mean([y[1] for y in face_outline_cluster])

            im = im.crop(left=leftmost,right=rightmost, top=y_median-STANDARD_FACE_HEIGHT/2, bottom=y_median+STANDARD_FACE_HEIGHT/2)

            # Create image with bounding boxes

            bbox_im = im.create_bbox_image()
            bboxed_im = bbox_im.pad_bbox('left_eye', (20, 40, 20, 40)).pad_bbox('right_eye', (20, 40, 20, 40))

            # Create ColumnStruct
            orig_struct = ColumnStruct (filename = orig_fname, header=("filename", "height", "left", "top", "width", "label"), label_by_index=True)
            orig_struct.add(list(bboxed_im.bbox['left_eye']), 1)
            orig_struct.add(list(bboxed_im.bbox['right_eye']), 1)
            combined_container.append(orig_struct.entry())

            bboxed_im.image.save(DESTINATION_PATH / tset / orig_fname)

            print(os.path.basename(fl_image.fpath))

    matrix = {'digitStruct' : np.array([combined_container], dtype=[('name', 'O'), ('bbox', 'O')])}
    hdf5storage.write(matrix, '.', str(DESTINATION_PATH / tset / 'digitStruct.mat') , matlab_compatible=True)

image_0001.png
image_0002.png
image_0003.png
image_0004.png
image_0005.png
image_0006.png
image_0007.png
image_0008.png
image_0009.png
image_0011.png
image_0012.png
image_0013.png
image_0014.png
image_0015.png
image_0016.png
image_0018.png
image_0021.png
image_0022.png
image_0023.png
image_0024.png
image_0025.png
image_0026.png
image_0027.png
image_0028.png
image_0029.png
image_0030.png
image_0031.png
image_0032.png
image_0033.png
image_0034.png
image_0035.png
image_0036.png
image_0037.png
image_0038.png
image_0039.png
image_0040.png
image_0041.png
image_0042.png
image_0043.png
image_0044.png
image_0045.png
image_0046.png
image_0048.png
image_0049.png
image_0050.png
image_0051.png
image_0052.png
image_0053.png
image_0054.png
image_0055.png
image_0056.png


KeyboardInterrupt: 